In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
import time
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from collections import defaultdict


# Set up the WebDriver (make sure to specify your path if needed)
driver = webdriver.ChromiumEdge()  # or webdriver.Firefox()
try:
    # Open the main page
    driver.get("https://mpbhulekh.gov.in/")

    # Wait for the page to load
    time.sleep(10)

    # Handle any alerts if present
    try:
        Alert(driver).accept()
    except:
        pass

    bhu_abhilekh = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//b[text()='भू-अभिलेख']"))
    )
    driver.execute_script("arguments[0].click();", bhu_abhilekh)


    # Click the "Yes" button on the pop-up
    try:
        yes_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "khasraNew"))
        )
        yes_button.click()
    except Exception as e:
        print("Error occurred:", e)

    # Wait for the new content to load
    time.sleep(5)

    # Wait for the iframe to load and switch to it
    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "MyFrame"))
    )

    # Locate the ddlDistrictId dropdown within the iframe
    district_dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ddlDistrictId"))
    )

    # Get all option values from the dropdown
    select = Select(district_dropdown)
    district_tuples = [
        (option.get_attribute('value'), option.text)
        for option in select.options if option.get_attribute != '0'
        ]

    # Print the tuples containing value, English name, and Hindi name
    print(district_tuples)

    #selecting first Distrct based on requirements -
    select.select_by_index(1)  # Select the first actual district
    teshil_dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ddlTehsilId"))
    )
    #Fetechil All Teshil - Selcted District
    teshil_All = teshil_dropdown.find_elements(By.TAG_NAME, 'option')
    teshil_tuples = [(option.text, option.get_attribute('value')) for option in teshil_All if option.get_attribute('value') != '0']

except Exception as e:
    print(f'Error - {e}')

## Find The Teshil followed by District and Village --> Select Khasra

In [ ]:
result = defaultdict(list)

#Selct Teshil Drop Down to Proceed Next 
temp = Select(teshil_dropdown)

#here we're using while loop purpose - data not come istaed of while we can use time.sleep after selecting teshil to wait to load village dropdown 
k = 0
while k < len(teshil_tuples):
    tehsil_name = teshil_tuples[k][0]  # Assuming the second item is the name
    # Select the tehsil by its visible text
    temp.select_by_visible_text(tehsil_name)

    # Wait for the village dropdown to load
    village_dropdown = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "ddlVillage"))
    )
    # time.sleep(2)
    # Get the village options
    kkVillage = village_dropdown.find_elements(By.TAG_NAME, 'option')
    if len(kkVillage) > 1:
        k += 1
        #gathering all village info for selcted teshil
        villageName = [(option.text, option.get_attribute('value')) for option in kkVillage if option.get_attribute('value') != '0']
        result[tehsil_name].append(villageName)
        #now to find khasra for particular village need to select village dropdown
        tempVillage = Select(village_dropdown)
        for i in villageName:
            #selction by name of village in drop down
            tempVillage.select_by_visible_text(i[0])
            #radio button click to show khasra dropdown for selected village
            select_t2_radio = driver.find_element(By.ID, "selectT2")
            select_t2_radio.click()
            try:
                #gathering all khasra number
                khasra = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.ID, "ddlKhSankhya"))
                    )
                #to load for wait 2 sec
                time.sleep(2)
                khasraOptions = khasra.find_elements(By.TAG_NAME, 'option')
                result[tehsil_name].append(len(khasraOptions))
                #result[tehsil_name].append([(jj.text) for jj in khasraOptions])
            except Exception as e:
                print('Error - ', e)



### For Generating the Excel Sheet

In [ ]:
#for printing the excel sheet
import openpyxl

# Create a new workbook and select the active sheet
workbook = openpyxl.Workbook()
sheet = workbook.active

# Set the title of the sheet
sheet.title = "MP Scrap"

# Write headers
sheet.append(["District Name", "Teshil Name", "Village", "Village Code", "Khasra Count"])

#result is  a dict every key is teshil name follwed by one list first 0th index elemt is village name - follwed by all khasra count

# Write data to the sheet
for teshil in result:
        for vill in range(len(result[teshil][0])):
            sheet.append(['Agar-Malwa | आगर-मालवा', teshil, *result[teshil][0][vill], result[teshil][vill + 1]])

# Save the workbook to a file
workbook.save("tehsil_villages.xlsx")
